In [2]:
import os
import torch
import re 
import datetime
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2
sys.path.append("..")


In [11]:
# const
# 毛色
dict_coat_color = {'栃栗': '1', '栗毛': '2', '白毛': '3', '芦毛': '4', '青毛': '5', '青鹿': '6', '鹿毛': '7', '黒鹿': '8'}
# 生産地 from csv to dict
#
# 場所名		
dict_place = {'札幌': '01', '函館': '02', '福島': '03', '新潟': '04', '東京': '05', '中山': '06', '中京': '07', '京都': '08', '阪神': '09', '小倉': '10'}

# クラスコード
# 未勝利,未出走,新馬,400万下,500万下,900万下,1000万下,1500万下,1600万下,重賞以外のオープン,グレード無し重賞,G3,G2,G1
dict_class_code = {'7': '01', '11': '02', '15': '03', '19': '04', '23': '05', '39': '06', '43': '07', '63': '08', '67': '09', '115': '10', '131': '11', '147': '12', '163': '13', '179': '14', '195': '15'}

# グレードコード
dict_grade_code = {'-': '1', 'A': '2', 'B': '3', 'C': '4', 'D': '5', 'E': '6', 'L': '7'}

# トラックコード
dict_track_code = {'0': '1', '1': '2', '8': '3'}

# コース区分
dict_course_division = {'-': '1', 'A': '2', 'B': '3', 'C': '4', 'D': '5', 'A1': '6', 'A2': '7'}

# 馬場状態
dict_ground_status = {'不': '1', '稍': '2', '良': '3', '重': '4'}

# 天候
dict_weather = {'小雨': '1', '小雪': '2', '晴': '3', '曇': '4', '雨': '5', '雪': '6'}

# 馬　性別
dict_gender = {'セ': '1', '牝': '2', '牡': '3'}

# 毛色
dict_coat_color = {'栃栗': '1', '栗毛': '2', '白毛': '3', '芦毛': '4', '青毛': '5', '青鹿': '6', '鹿毛': '7', '黒鹿': '8'}

# 父タイプ名
dict_horse_type = {'-': 0, 'その他のエクリプス系': '1', 'セントサイモン系': '2', 'トゥルビヨン系': '3', 'ナスルーラ系': '4', 'ニアークティック系': '5', 'ネイティヴダンサー系': '6', 'マンノウォー系': '7', 'ロイヤルチャージャー系': '8'}

# 騎手_所属
dict_rider_belong = {'地方': '1', '外国': '2', '栗東': '3', '美浦': '4'}


In [12]:
# ------------------------------------
# 　成績
# ------------------------------------

# train orignal data
# csv_file = os.path.join('../data/csv/', '成績.csv')
# val orignal data
csv_file = os.path.join('../data/csv/', '成績_2021.csv')

df_score = pd.read_csv(csv_file, 
dtype={'年': str, '月': str, '日': str, '日次': str, 'レース番号': str,
       'コーナー回数': str, '頭数': str, 'PCI': str, 'RPCI': str,
       'PCI3': str, '馬番': str, '斤量': str, '単勝オッズ': str,
       '走破タイム（秒）': str}).add_prefix('成績_')

# replace none data
df_score.fillna('-', inplace=True)

# 障害競走データ除く
df_score = df_score.loc[df_score['成績_トラックコード'].isin([0, 1, 8])]

# df_score['成績_年'] 変更なし
# df_score['成績_月'] 変更なし
# df_score['成績_日'] 変更なし
# df_score['成績_回次'] 変更なし
df_score['成績_場所'] = df_score['成績_場所'].apply(lambda x: dict_place[str(x)])
df_score['成績_日次'] = df_score['成績_日次'].replace({'A':'10', 'B':'11', 'C': '12'}).apply(lambda x: x.zfill(2))
df_score['成績_レース番号'] = df_score['成績_レース番号'].apply(lambda x: x.zfill(2))
df_score['成績_クラスコード'] = df_score['成績_クラスコード'].apply(lambda x: dict_class_code[str(x)])
df_score['成績_グレードコード'] = df_score['成績_グレードコード'].apply(lambda x: dict_grade_code[str(x)])
df_score['成績_トラックコード'] = df_score['成績_トラックコード'].apply(lambda x: dict_track_code[str(x)])
# df_score['成績_コーナー回数'] 変更なし
df_score['成績_距離'] = df_score['成績_距離'] // 100
df_score['成績_コース区分'] = df_score['成績_コース区分'].apply(lambda x: dict_course_division[str(x)])
df_score['成績_馬場状態'] = df_score['成績_馬場状態'].apply(lambda x: dict_ground_status[str(x)])
df_score['成績_天候'] = df_score['成績_天候'].apply(lambda x: dict_weather[str(x)])
df_score['成績_頭数'] = df_score['成績_頭数'].apply(lambda x: x.zfill(2))
df_score['成績_発走時刻'] = df_score['成績_発走時刻'].str.replace(':', '')
df_score['成績_PCI'] = df_score['成績_PCI'].str.replace('.', '').replace('-', '0').apply(lambda x: x.zfill(4))
df_score['成績_RPCI'] = df_score['成績_RPCI'].str.replace('.', '').replace('-', '0').apply(lambda x: x.zfill(4))
df_score['成績_PCI3'] = df_score['成績_PCI3'].str.replace('.', '').replace('-', '0').apply(lambda x: x.zfill(4))
df_score['成績_馬番'] = df_score['成績_馬番'].apply(lambda x: x.zfill(2))
# df_score['成績_枠番'] 変更なし
df_score['成績_斤量'] = df_score['成績_斤量'].str.replace('.', '').str.zfill(2)
df_score['成績_人気'] = df_score['成績_人気'].replace('-', 0).astype(np.int64).astype(str).apply(lambda x: x.zfill(2))
# odds to val model
df_score['成績_val_単勝オッズ'] = df_score['成績_単勝オッズ'].str.replace('-', '0')
df_score['成績_単勝オッズ'] = df_score['成績_単勝オッズ'].str.replace('.', '').replace('-', '0').apply(lambda x: x.zfill(4))
df_score['成績_走破タイム（秒）'] = df_score['成績_走破タイム（秒）'].str.replace('.', '').replace('-', '0').apply(lambda x: x.zfill(4))
df_score['成績_馬体重'] = df_score['成績_馬体重'].replace('-', 0).astype(np.int64).astype(str).apply(lambda x: x.zfill(3))

df_score['成績_血統登録番号'] = df_score['成績_血統登録番号'].str.slice(start=4).fillna('00000000')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single char

In [13]:
# ------------------------------------
# 　馬　
# ------------------------------------
csv_file = os.path.join('../data/csv/', '馬.csv')
df_horse = pd.read_csv(csv_file, dtype={'馬主コード': str, '年齢': str, '収得賞金': str, '本賞金': str, '生年': str}).add_prefix('馬_')
df_horse['馬_馬主コード'] = 'OX' + df_horse['馬_馬主コード']
df_horse.fillna('-', inplace=True)

df_horse_merage = pd.read_csv(csv_file).add_prefix('merage_')

csv_file = os.path.join('../data/csv/', '厩舎.csv')
df_stable = pd.read_csv(csv_file, dtype={'ID': str})

csv_file = os.path.join('../data/csv/', '生産地.csv')
df_place = pd.read_csv(csv_file, dtype={'ID': str})


df_horse['馬_性別'] = df_horse['馬_性別'].apply(lambda x: dict_gender[x])
df_horse['馬_年齢'] = df_horse['馬_年齢'].apply(lambda x: x.zfill(2))

merged_stable = pd.merge(df_horse, df_stable, left_on='馬_厩舎', right_on='厩舎', how='left')
df_horse['馬_厩舎'] = merged_stable['ID'].fillna('0000').str.zfill(4)

merged = pd.merge(df_horse, df_horse_merage, left_on='馬_種牡馬', right_on='merage_馬名', how='left')
df_horse['馬_種牡馬'] = merged['merage_血統登録番号'].str.slice(start=4).fillna('00000000')

merged = pd.merge(df_horse, df_horse_merage, left_on='馬_母名', right_on='merage_馬名', how='left')
df_horse['馬_母名'] = merged['merage_血統登録番号'].str.slice(start=4).fillna('00000000')

merged = pd.merge(df_horse, df_horse_merage, left_on='馬_母父名', right_on='merage_馬名', how='left')
df_horse['馬_母父名'] = merged['merage_血統登録番号'].str.slice(start=4).fillna('00000000')

merged = pd.merge(df_horse, df_horse_merage, left_on='馬_母の母名', right_on='merage_馬名', how='left')
df_horse['馬_母の母名'] = merged['merage_血統登録番号'].str.slice(start=4).fillna('00000000')

merged = pd.merge(df_horse, df_horse_merage, left_on='馬_母の母の父名', right_on='merage_馬名', how='left')
df_horse['馬_母の母の父名'] = merged['merage_血統登録番号'].str.slice(start=4).fillna('00000000')

merged = pd.merge(df_horse, df_horse_merage, left_on='馬_母の母の母', right_on='merage_馬名', how='left')
df_horse['馬_母の母の母'] = merged['merage_血統登録番号'].str.slice(start=4).fillna('00000000')

df_horse['馬_毛色'] = df_horse['馬_毛色'].apply(lambda x: dict_coat_color[x])
df_horse['馬_収得賞金'] = df_horse['馬_収得賞金'].apply(lambda x: x.zfill(8))
df_horse['馬_本賞金'] = df_horse['馬_本賞金'].apply(lambda x: x.zfill(8))

def join_arrivals(x):
    data = re.split('- |-', x)
    result = []
    for item in data:
        item = '99' if len(item) > 2 else item
        result.append(item.strip().zfill(2))
    return str(''.join(result))
df_horse['馬_着度数'] = df_horse['馬_着度数'].apply(join_arrivals)

merged_place = pd.merge(df_horse, df_place, left_on='馬_生産地', right_on='生産地', how='left')
df_horse['馬_生産地'] = merged_place['ID'].fillna('000').str.zfill(3)

df_horse['馬_生年'] = df_horse['馬_生年'].str.slice(start=2)
df_horse['馬_誕生日'] = df_horse['馬_誕生日'].str.split('月|日').apply(lambda x: ''.join([str(item.strip().zfill(2)) for item in x[:2]]))
df_horse['馬_血統登録番号'] = df_horse['馬_血統登録番号'].str.slice(start=4).fillna('00000000')
df_horse['馬_取引価格'] = df_horse['馬_取引価格'].str.replace('万円|\(他\)', '', regex=True).replace('-', '0', regex=False).astype(float).apply(lambda x: str(int(x * 10000)).zfill(9))
df_horse['馬_父タイプ名'] = df_horse['馬_父タイプ名'].apply(lambda x: dict_horse_type[x])
df_horse['馬_母父タイプ名'] = df_horse['馬_母父タイプ名'].apply(lambda x: dict_horse_type[x])
df_horse['馬_母母父タイプ名'] = df_horse['馬_母母父タイプ名'].apply(lambda x: dict_horse_type[x])


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (35,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
# ------------------------------------
# 　騎手
# ------------------------------------
csv_file = os.path.join('../data/csv/', '騎手.csv')
df_rider = pd.read_csv(csv_file, dtype=str).add_prefix('騎手_')
df_rider.fillna('-', inplace=True)

df_rider['騎手_順位'] = df_rider['騎手_順位'].apply(lambda x: x.zfill(4))
df_rider['騎手_所属'] = df_rider['騎手_所属'].apply(lambda x: dict_rider_belong[x])
df_rider['騎手_誕生日'] = df_rider['騎手_誕生日'].str.split('\.').apply(lambda x: ''.join([str(item[-2:].strip().replace('-', '').zfill(2)) for item in x]).zfill(6))
df_rider['騎手_免許取得年'] = df_rider['騎手_免許取得年'].str.slice(start=2)
df_rider['騎手_キャリア'] = df_rider['騎手_キャリア'].str.zfill(2)
df_rider['騎手_1着数'] = df_rider['騎手_1着数'].str.zfill(5)
df_rider['騎手_2着数'] = df_rider['騎手_2着数'].str.zfill(5)
df_rider['騎手_3着数'] = df_rider['騎手_3着数'].str.zfill(5)
df_rider['騎手_着外数'] = df_rider['騎手_着外数'].str.zfill(5)
df_rider['騎手_前年1着'] = df_rider['騎手_前年1着'].str.zfill(5)
df_rider['騎手_前年2着'] = df_rider['騎手_前年2着'].str.zfill(5)
df_rider['騎手_前年3着'] = df_rider['騎手_前年3着'].str.zfill(5)
df_rider['騎手_前年着外'] = df_rider['騎手_前年着外'].str.zfill(5)
df_rider['騎手_生涯1着'] = df_rider['騎手_生涯1着'].str.zfill(5)
df_rider['騎手_生涯2着'] = df_rider['騎手_生涯2着'].str.zfill(5)
df_rider['騎手_生涯3着'] = df_rider['騎手_生涯3着'].str.zfill(5)
df_rider['騎手_生涯着外'] = df_rider['騎手_生涯着外'].str.zfill(5)
df_rider['騎手_コード'] = df_rider['騎手_コード'].str.slice(start=3)
df_rider['騎手_前年順位(平地)'] = df_rider['騎手_前年順位(平地)'].str.zfill(3)

In [15]:
# ------------------------------------
# 　調教師
# ------------------------------------
csv_file = os.path.join('../data/csv/', '調教師.csv')
df_trainer = pd.read_csv(csv_file, dtype=str).add_prefix('調教師_')
df_trainer.fillna('', inplace=True)

df_trainer['調教師_順位'] = df_trainer['調教師_順位'].apply(lambda x: x.zfill(4))
df_trainer['調教師_所属'] = df_trainer['調教師_所属'].apply(lambda x: dict_rider_belong[x])
df_trainer['調教師_誕生日'] = df_trainer['調教師_誕生日'].str.split('\.').apply(lambda x: ''.join([str(item[-2:].strip().replace('-', '').zfill(2)) for item in x]).zfill(6))
df_trainer['調教師_免許取得年'] = df_trainer['調教師_免許取得年'].str.slice(start=2).str.zfill(2)
df_trainer['調教師_キャリア'] = df_trainer['調教師_キャリア'].str.zfill(2)
df_trainer['調教師_1着数'] = df_trainer['調教師_1着数'].str.zfill(5)
df_trainer['調教師_2着数'] = df_trainer['調教師_2着数'].str.zfill(5)
df_trainer['調教師_3着数'] = df_trainer['調教師_3着数'].str.zfill(5)
df_trainer['調教師_着外数'] = df_trainer['調教師_着外数'].str.zfill(5)
df_trainer['調教師_前年1着'] = df_trainer['調教師_前年1着'].str.zfill(5)
df_trainer['調教師_前年2着'] = df_trainer['調教師_前年2着'].str.zfill(5)
df_trainer['調教師_前年3着'] = df_trainer['調教師_前年3着'].str.zfill(5)
df_trainer['調教師_前年着外'] = df_trainer['調教師_前年着外'].str.zfill(5)
df_trainer['調教師_生涯1着'] = df_trainer['調教師_生涯1着'].str.zfill(5)
df_trainer['調教師_生涯2着'] = df_trainer['調教師_生涯2着'].str.zfill(5)
df_trainer['調教師_生涯3着'] = df_trainer['調教師_生涯3着'].str.zfill(5)
df_trainer['調教師_生涯着外'] = df_trainer['調教師_生涯着外'].str.zfill(5)
df_trainer['調教師_コード'] = df_trainer['調教師_コード'].str.slice(start=3)
df_trainer['調教師_前年順位(平地)'] = df_trainer['調教師_前年順位(平地)'].str.zfill(3)

In [16]:
# ------------------------------------
# 　馬主
# ------------------------------------
csv_file = os.path.join('../data/csv/', '馬主.csv')
df_horse_owner = pd.read_csv(csv_file, dtype=str).add_prefix('馬主_')
df_horse_owner.fillna('-', inplace=True)

df_horse_owner['馬主_馬主コード'] = df_horse_owner['馬主_馬主コード'].str.slice(start=2)
df_horse_owner['馬主_本年1着'] = df_horse_owner['馬主_本年1着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_本年2着'] = df_horse_owner['馬主_本年2着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_本年3着'] = df_horse_owner['馬主_本年3着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_本年4着'] = df_horse_owner['馬主_本年4着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_本年5着'] = df_horse_owner['馬主_本年5着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_本年6着以下'] = df_horse_owner['馬主_本年6着以下'].str.replace('-', '0').str.zfill(5)

df_horse_owner['馬主_本年本賞金計(万円)'] = df_horse_owner['馬主_本年本賞金計(万円)'].str.replace('-', '0').str.zfill(8)
df_horse_owner['馬主_本年1走当本賞金(万円)'] = df_horse_owner['馬主_本年1走当本賞金(万円)'].str.replace('-', '0').str.zfill(8)

df_horse_owner['馬主_通算1着'] = df_horse_owner['馬主_通算1着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_通算2着'] = df_horse_owner['馬主_通算2着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_通算3着'] = df_horse_owner['馬主_通算3着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_通算4着'] = df_horse_owner['馬主_通算4着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_通算5着'] = df_horse_owner['馬主_通算5着'].str.replace('-', '0').str.zfill(5)
df_horse_owner['馬主_通算6着以下'] = df_horse_owner['馬主_通算6着以下'].str.replace('-', '0').str.zfill(5)

df_horse_owner['馬主_通算本賞金計(万円)'] = df_horse_owner['馬主_通算本賞金計(万円)'].str.replace('-', '0').str.zfill(8)
df_horse_owner['馬主_通算1走当本賞金(万円)'] = df_horse_owner['馬主_通算1走当本賞金(万円)'].str.replace('-', '0').str.zfill(8)

In [17]:
# ------------------------------------
# 　生産者
# ------------------------------------
csv_file = os.path.join('../data/csv/', '生産者_drop_duplicates.csv')
df_producer = pd.read_csv(csv_file, dtype=str).add_prefix('生産者_')
df_producer.fillna('-', inplace=True)

df_producer['生産者_生産者コード'] = df_producer['生産者_生産者コード'].str.slice(start=2)
df_producer['生産者_本年1着'] = df_producer['生産者_本年1着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_本年2着'] = df_producer['生産者_本年2着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_本年3着'] = df_producer['生産者_本年3着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_本年4着'] = df_producer['生産者_本年4着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_本年5着'] = df_producer['生産者_本年5着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_本年6着以下'] = df_producer['生産者_本年6着以下'].str.replace('-', '0').str.zfill(5)

df_producer['生産者_本年本賞金計(万円)'] = df_producer['生産者_本年本賞金計(万円)'].str.replace('-', '0').str.zfill(8)
df_producer['生産者_本年1走当本賞金(万円)'] = df_producer['生産者_本年1走当本賞金(万円)'].str.replace('-', '0').str.zfill(8)

df_producer['生産者_通算1着'] = df_producer['生産者_通算1着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_通算2着'] = df_producer['生産者_通算2着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_通算3着'] = df_producer['生産者_通算3着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_通算4着'] = df_producer['生産者_通算4着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_通算5着'] = df_producer['生産者_通算5着'].str.replace('-', '0').str.zfill(5)
df_producer['生産者_通算6着以下'] = df_producer['生産者_通算6着以下'].str.replace('-', '0').str.zfill(5)

df_producer['生産者_通算本賞金計(万円)'] = df_producer['生産者_通算本賞金計(万円)'].str.replace('-', '0').str.zfill(8)
df_producer['生産者_通算1走当本賞金(万円)'] = df_producer['生産者_通算1走当本賞金(万円)'].str.replace('-', '0').str.zfill(8)


In [18]:
# merge info
# 出馬表・成績(血統登録番号)     馬(血統登録番号)
merged = pd.merge(df_score, df_horse, left_on='成績_血統登録番号', right_on='馬_血統登録番号', how='left')

# 出馬表・成績(騎手コード)       騎手(コード)
merged = pd.merge(merged, df_rider, left_on='成績_騎手コード', right_on='騎手_コード',how='left')

# 出馬表・成績(調教師コード)     調教師(コード)
merged = pd.merge(merged, df_trainer, left_on='成績_調教師コード', right_on='調教師_コード',how='left')

# 馬(馬主コード)                馬主(馬主コード)
merged = pd.merge(merged, df_horse_owner, left_on='馬_馬主コード', right_on='馬主_馬主コード',how='left')

# 馬(生産者)                    生産者(生産者名)
merged = pd.merge(merged, df_producer, left_on='馬_生産者', right_on='生産者_生産者名',how='left')

merged = merged[['成績_レースID（新）', '成績_入線着順', '成績_val_単勝オッズ', '成績_年', '成績_月', '成績_日', '成績_回次', '成績_場所', '成績_日次', 
'成績_レース番号', '成績_クラスコード', '成績_グレードコード', '成績_トラックコード', '成績_コーナー回数', '成績_距離', 
'成績_コース区分', '成績_馬場状態', '成績_天候', '成績_頭数', '成績_発走時刻', '成績_PCI', '成績_RPCI', '成績_PCI3', 
'成績_馬番', '成績_枠番', '成績_斤量', '成績_人気', '成績_単勝オッズ', '成績_走破タイム（秒）', '成績_馬体重', '馬_性別',
'馬_年齢', '馬_厩舎', '馬_種牡馬', '馬_母名', '馬_母父名', '馬_母の母名', '馬_母の母の父名', '馬_母の母の母', '馬_毛色', 
'馬_収得賞金', '馬_本賞金', '馬_着度数', '馬_生産地', '馬_生年', '馬_誕生日', '馬_血統登録番号', '馬_取引価格', '馬_父タイプ名', 
'馬_母父タイプ名', '馬_母母父タイプ名', '騎手_順位', '騎手_所属', '騎手_誕生日', '騎手_免許取得年', '騎手_キャリア', '騎手_1着数', 
'騎手_2着数', '騎手_3着数', '騎手_着外数', '騎手_前年1着', '騎手_前年2着', '騎手_前年3着', '騎手_前年着外', '騎手_生涯1着', 
'騎手_生涯2着', '騎手_生涯3着', '騎手_生涯着外', '騎手_コード', '騎手_前年順位(平地)', 
'調教師_順位', '調教師_所属', '調教師_誕生日', '調教師_免許取得年', '調教師_キャリア', '調教師_1着数', '調教師_2着数', '調教師_3着数', 
'調教師_着外数', '調教師_前年1着', '調教師_前年2着', '調教師_前年3着', '調教師_前年着外', '調教師_生涯1着', '調教師_生涯2着', 
'調教師_生涯3着', '調教師_生涯着外', '調教師_コード', '調教師_前年順位(平地)', '馬主_馬主コード', '馬主_本年1着', '馬主_本年2着', 
'馬主_本年3着', '馬主_本年4着', '馬主_本年5着', '馬主_本年6着以下', '馬主_本年本賞金計(万円)', '馬主_本年1走当本賞金(万円)', 
'馬主_通算1着', '馬主_通算2着', '馬主_通算3着', '馬主_通算4着', '馬主_通算5着', '馬主_通算6着以下', '馬主_通算本賞金計(万円)', 
'馬主_通算1走当本賞金(万円)', '生産者_生産者コード', '生産者_本年1着', '生産者_本年2着', '生産者_本年3着', '生産者_本年4着', 
'生産者_本年5着', '生産者_本年6着以下', '生産者_本年本賞金計(万円)', '生産者_本年1走当本賞金(万円)', '生産者_通算1着', 
'生産者_通算2着', '生産者_通算3着', '生産者_通算4着', '生産者_通算5着', '生産者_通算6着以下', '生産者_通算本賞金計(万円)', '生産者_通算1走当本賞金(万円)']]

# None to 0
values = {'馬_性別': '0', '馬_年齢': '00', '馬_厩舎': '0000', '馬_種牡馬': '00000000', '馬_母名': '00000000', 
'馬_母父名': '00000000', '馬_母の母名': '00000000', '馬_母の母の父名': '00000000', '馬_母の母の母': '00000000', 
'馬_毛色': '0', '馬_収得賞金': '00000000', '馬_本賞金': '00000000', '馬_着度数': '00000000', '馬_生産地': '000', 
'馬_生年': '00', '馬_誕生日': '0000', '馬_血統登録番号': '00000000', '馬_取引価格': '00000000', 
'馬_父タイプ名': '0', '馬_母父タイプ名': '0', '馬_母母父タイプ名': '0', '騎手_順位': '0000', '騎手_所属': '0', 
'騎手_誕生日': '000000', '騎手_免許取得年': '00', '騎手_キャリア': '00', '騎手_1着数': '00000', '騎手_2着数': '00000', 
'騎手_3着数': '00000', '騎手_着外数': '0000', '騎手_前年1着': '00000', '騎手_前年2着': '00000', '騎手_前年3着': '00000', 
'騎手_前年着外': '00000', '騎手_生涯1着': '00000', '騎手_生涯2着': '00000', '騎手_生涯3着': '00000', 
'騎手_生涯着外': '00000', '騎手_コード': '0000', '騎手_前年順位(平地)': '000', '調教師_順位': '0000', 
'調教師_所属': '0', '調教師_誕生日': '000000', '調教師_免許取得年': '00', '調教師_キャリア': '00', 
'調教師_1着数': '00000', '調教師_2着数': '00000', '調教師_3着数': '00000', '調教師_着外数': '00000', '調教師_前年1着': '00000', 
'調教師_前年2着': '00000', '調教師_前年3着': '00000', '調教師_前年着外': '00000', '調教師_生涯1着': '00000', 
'調教師_生涯2着': '00000', '調教師_生涯3着': '00000', '調教師_生涯着外': '00000', '調教師_コード': '0000', 
'調教師_前年順位(平地)': '000', '馬主_馬主コード': '000000', '馬主_本年1着': '00000', '馬主_本年2着': '00000', 
'馬主_本年3着': '00000', '馬主_本年4着': '00000', '馬主_本年5着': '00000', '馬主_本年6着以下': '00000', 
'馬主_本年本賞金計(万円)': '00000000', '馬主_本年1走当本賞金(万円)': '00000000', '馬主_通算1着': '00000', 
'馬主_通算2着': '00000', '馬主_通算3着': '00000', '馬主_通算4着': '00000', '馬主_通算5着': '00000', 
'馬主_通算6着以下': '00000', '馬主_通算本賞金計(万円)': '00000000', '馬主_通算1走当本賞金(万円)': '00000000', 
'生産者_生産者コード': '000000', '生産者_本年1着': '00000', '生産者_本年2着': '00000', '生産者_本年3着': '00000', 
'生産者_本年4着': '00000', '生産者_本年5着': '00000', '生産者_本年6着以下': '00000', '生産者_本年本賞金計(万円)': '00000000', 
'生産者_本年1走当本賞金(万円)': '00000000', '生産者_通算1着': '00000', '生産者_通算2着': '00000', 
'生産者_通算3着': '00000', '生産者_通算4着': '00000', '生産者_通算5着': '00000', '生産者_通算6着以下': '00000', 
'生産者_通算本賞金計(万円)': '00000000', '生産者_通算1走当本賞金(万円)': '00000000'}
merged.fillna(value=values, inplace=True)

merged = merged.astype(str)
merged.sort_values(by='成績_レースID（新）', inplace=True)

In [ ]:
# generate train dataset
merged.to_csv('../data/csv/result_train_merged.csv', index=False)
pd.concat([merged[['成績_レースID（新）', '成績_入線着順']], merged.iloc[:, 3:].agg(''.join, axis=1)], axis=1).to_csv('../data/csv/train.csv', index=False)

In [23]:
# generate val dataset
merged.to_csv('../data/csv/result_val_merged.csv', index=False)
merged = pd.concat([merged[['成績_レースID（新）', '成績_入線着順', '成績_val_単勝オッズ']], merged.iloc[:, 3:].agg(''.join, axis=1)], axis=1)
merged = merged.rename(columns={'成績_レースID（新）': 'race_id', '成績_入線着順': 'score', '成績_val_単勝オッズ': 'odds', 0: 'baseinfo'}, index={'ONE': 'Row_1'})
merged.to_csv('../data/csv/val.csv', index=False)

In [ ]:
pd.set_option('display.max_rows', 40)
#merged = merged.astype(str)
#print(merged.dtypes)

In [ ]:
# ------------------------------------
# 　生産者 data drop_duplicates, do not delete this
# ------------------------------------

csv_file = os.path.join('../data/csv/', '生産者.csv')
df_producer = pd.read_csv(csv_file)
df_producer_no_d = pd.read_csv(csv_file, usecols=['生産者コード', '生産者名']).add_prefix('生産者_nod_')

df_producer = df_producer.groupby(['生産者名']).sum().reset_index()
df_producer_no_d.drop_duplicates(subset='生産者_nod_生産者名', inplace=True)
result = pd.merge(df_producer_no_d, df_producer, left_on='生産者_nod_生産者名', right_on='生産者名',how='left')
result = result.rename(columns={'生産者_nod_生産者コード': '生産者コード'})
result.iloc[:, 3:] = result.iloc[:, 3:].astype(int)
result.to_csv('../data/csv/生産者_drop_duplicates.csv', index=False)

In [ ]:
# test result data
csv_file = os.path.join('../data/csv/', 'val.csv')
data_original = pd.read_csv(csv_file, dtype=str)
data_original['baseinfo'].str.len().hist()

In [ ]:
# data.groupby('成績_レースID（新）').apply(lambda x: x.sum())
targets = data_original.groupby('race_id')['score'].apply(lambda x: np.pad(x.values, (0, 18 - x.values.shape[0]), 'constant', constant_values='0')).to_numpy()
targets = np.stack(targets, axis=0).astype(np.uint8)

In [ ]:
targets.shape

In [ ]:
# data_original['baseinfo'] = data_original['baseinfo'].apply(lambda x: np.array(list(x)))
# constant_values = ''.zfill(576)
data = data_original.groupby('race_id')['baseinfo'].apply(lambda x: ''.join(x).ljust(576 * 18, '0')).to_numpy()
data

In [ ]:
for idx, x in np.ndenumerate(data):
    #print(len(x))
    #print(idx)
    data[idx] = np.asarray(list(x), dtype=np.uint8)
    # break
data = np.stack(data, axis=0)

In [ ]:
data[0].shape

In [ ]:
a24 = np.reshape(data[0], (24, 24, 18))
a24 = a24[:, :, :3]
# a24.shape
b = cv2.resize(a24, (55, 55))
print(b.shape)
t(b)
plt.imshow(t0(t(b)))

In [ ]:
t0 = transforms.ToPILImage()
# t = transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
t = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
image = cv2.imread("E:/datasets/anime-faces/data1/495.png")   # when reading the image the image original size is 150x150
print(image.shape)
scaled_image = cv2.resize(image, (24, 24))  # when scaling we scale original image to 24x24 
print(scaled_image.shape)

In [ ]:
a24 = np.random.randint(0, 10, size=(24, 24, 3), dtype=np.uint8)
b = cv2.resize(a24, (55, 55))
print(b.shape)
t(b)

In [9]:
'odds' in data_original.columns

False

In [3]:
data_original = pd.read_csv(os.path.join('../data/csv/', 'val.csv'), dtype=str)
data_original['race_id'] = data_original['race_id'].str[:-2]
data_original['baseinfo'] = data_original['baseinfo'].str.ljust(576, '0')

data_original['score'] = data_original['score'].astype('int64')
targets = data_original.groupby('race_id')['score'].apply(lambda x: np.pad(x.values, (0, 18 - x.values.shape[0]), 'constant', constant_values='0')).to_numpy()
targets = np.stack(targets, axis=0)

targets

array([[11,  7,  3, ...,  5,  0,  0],
       [ 6, 10,  4, ...,  1,  0,  0],
       [ 9, 10,  8, ...,  0,  0,  0],
       ...,
       [16, 15,  4, ..., 11,  0,  0],
       [ 1,  4,  3, ..., 11, 10,  0],
       [16,  1, 15, ...,  4,  0,  0]], dtype=int64)

In [5]:
data_original['odds'] = data_original['odds'].astype('float64')
odds = data_original.groupby('race_id')['odds'].apply(lambda x: np.pad(x.values, (0, 18 - x.values.shape[0]), 'constant', constant_values='0')).to_numpy()
odds = np.stack(odds, axis=0)
odds

array([[103. , 108.4,  13.6, ...,   9. ,   0. ,   0. ],
       [  2.8, 253.8,  55.6, ...,  22.7,   0. ,   0. ],
       [159.2,  43. ,  27.2, ...,   0. ,   0. ,   0. ],
       ...,
       [ 84.8, 286.1,   7.4, ...,  42.7,   0. ,   0. ],
       [  5.2,   3.8,   7.3, ..., 154.9,  12.2,   0. ],
       [ 82.1,  47. ,  74.4, ...,  94.6,   0. ,   0. ]])

In [34]:
data_original['score'] = data_original['score'].astype('int64')
# a = 0 if data_original['score'].astype('int64') > 3 else 1

In [39]:
data_original.groupby('race_id')['score'].apply(lambda x: np.pad(x.values, (0, 18 - x.values.shape[0]), 'constant', constant_values='0')).to_numpy()

array([array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64),
       array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int64),
       array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
       ...,
       array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int64),
       array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64),
       array([1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)],
      dtype=object)

In [38]:
data_original['score'] = data_original['score'].apply(lambda x: 0 if int(x) > 3 else 1)
data_original

,race_id,score,baseinfo
0,RX2001010506010101,0,0101051060101011221213316095504260394404701155...
1,RX2001010506010101,0,0101051060101011221213316095504040394404702153...
2,RX2001010506010101,1,0101051060101011221213316095504240394404703255...
3,RX2001010506010101,0,0101051060101011221213316095503900394404704255...
4,RX2001010506010101,0,0101051060101011221213316095504290394404705355...
...,...,...,...
951209,RX2020122709060812,0,2012276090812096121433314162505660522544310655...
951210,RX2020122709060812,0,2012276090812096121433314162505660522544311757...
951211,RX2020122709060812,0,2012276090812096121433314162505510522544312755...
951212,RX2020122709060812,0,2012276090812096121433314162505460522544313857...
